# Preprocessing data 

- This notebook consists of several preprocessing steps designed for a dataset, FinnSentiment, to be used for a text classification task.
- In order the text classification to work with some other dataset than FinnSentiment, you may need to tailor this notebook to fit your case. For example, FinnSentiment is originally a tsv file, but with a csv file you may load it with the following line:
```python
df = pd.read_csv(cfg.datafolder+"filename.csv")
```
- This notebook involves breaking down the data into multiple subsets, that are needed in later phases of the text classification. Separate sets are created for pretraining (MLM) and finetuning. Duplicate and NAN values are also checked and removed. 
- For running the text classification, or finetuning, the data should have annotations or labels that indicate a class for each data sample. For pretraining labels are omitted.
- The dataset presented here is openly available [FinnSentiment](https://korp.csc.fi/download/finsen/src/). Reference: \
Lindén, K., Jauhiainen, T., & Hardwick, S. (2020). FinnSentiment--A Finnish Social Media Corpus for Sentiment Polarity Annotation. arXiv preprint arXiv:2012.02613. https://arxiv.org/pdf/2012.02613.pdf

## 1. Import libraries, define configuration class, and helper functions

If some libraries are not installed, use command prompt with e.g.
```
pip install transformers
```

In [1]:
from transformers import AutoTokenizer
import numpy as np
import pandas as pd
from sklearn import model_selection
from sklearn.model_selection import train_test_split
import csv

In [2]:
class cfg():
    model_name = "TurkuNLP/bert-base-finnish-cased-v1" #"TurkuNLP/bert-large-finnish-cased-v1" 
    data_folder = "./data/" #alternatively use a full path: data_folder = "/path/to/data/"

In [3]:
#check how many samples there are for each class
def check_class_distribution(df, print_lengths=True):
    all_class_dist=[]
    for i in sorted(df.label.unique()):
        class_dist = len(df[df.label==i])
        if print_lengths:
            print('For label {0} there is {1} data samples'.format(i, class_dist))
        all_class_dist.append(class_dist)
    return all_class_dist

#check the length of data samples (each row in dataframe)
#length is calculated from tokenized form of text data (samples)
def check_token_length(df, print_lengths=True):
    tokenizer = AutoTokenizer.from_pretrained(cfg.model_name)
    x = df["text"].values
    
    # Encode our concatenated data
    encoded = [tokenizer.encode(sent, add_special_tokens=True) for sent in x]

    # Find the maximum, minimum, mean and median length
    t_lengths=[len(sent) for sent in encoded]
    max_len = max(t_lengths)
    mean_len = np.mean(t_lengths)
    median_len = np.median(t_lengths)
    min_len = min(t_lengths)
    
    if print_lengths:
        print('Min length: ', min_len)
        print('Mean length: ', mean_len)
        print('Median length: ', median_len)
        print('Max length: ', max_len)
    
    return min_len, median_len, mean_len, max_len

#creating k folds
def create_kfolds(data, num_splits, random_seed):
    data["kfold"] = -1
    kf = model_selection.KFold(n_splits=num_splits, shuffle=True, random_state=random_seed)
    for f, (t_, v_) in enumerate(kf.split(X=data)):
        data.loc[v_, 'kfold'] = f
    return data

#check and print the amount of duplicate and nan values in a dataframe
def check_dupl_nan(df):
    print('NaNs: ',df.isna().sum())
    for i in df.columns:
        print('Duplicates in {0}: {1}'.format(i,df[i].duplicated().sum()))

#remove duplicate and nan values in a dataframe
#nan values in nan_columns, duplicate values in dupl_columns
#note that this function was speficially created to edit one or two columns, may be irrelevant for other types of use
def remove_dupl_nan(df, nan_columns=['text'], dupl_columns=['text']):
    df = df.dropna(subset=nan_columns)
    df = df.drop_duplicates(subset=dupl_columns)
    df = df.reset_index(drop=True)
    return df

## 2. Changing data format and dropping irrelevant columns

In [4]:
d=[]

with open(cfg.data_folder+"FinnSentiment2020.tsv") as tsvfile:
    tsvreader = csv.reader(tsvfile, delimiter="\t", quoting=csv.QUOTE_NONE)
    for line in tsvreader:
        d.append(line)

In [5]:
df = pd.DataFrame(data=d, columns=['A_sentiment', 'B_sentiment', 'C_sentiment',
                                   'majority_value', 'derived_value', 'sentiment_smiley',
                                   'sentiment_productreview', 'split', 'batch',
                                   'orig_index', 'sentence_text'])
df

,A_sentiment,B_sentiment,C_sentiment,majority_value,derived_value,sentiment_smiley,sentiment_productreview,split,batch,orig_index,sentence_text
0,1,0,1,1,4,0,-1,1,1,comments2008c.vrt 2145269,- Tervetuloa skotlantiin...
1,0,1,0,0,4,0,-1,12,1,comments2011c.vrt 3247745,"...... No, oikein sopiva sattumaha se vaan oli..."
2,0,0,0,0,3,0,-1,14,1,comments2007c.vrt 3792960,40.
3,1,1,1,1,5,0,1,7,1,comments2010d.vrt 2351708,Kyseessä voi olla loppuelämäsi nainen.
4,1,1,1,1,5,0,1,12,1,comments2007d.vrt 1701675,Sinne vaan ocean clubiin iskemään!
...,...,...,...,...,...,...,...,...,...,...,...
26995,-1,0,0,0,2,0,1,15,9,threads2015a.vrt 5122042,sais nauraa.
26996,-1,0,-1,-1,2,0,-1,10,9,comments2015d.vrt 3337506,"Menkää töihin, Jonnekin muuannekin kun niihin ..."
26997,0,0,0,0,3,0,0,14,9,comments2011b.vrt 960278,Ja tiedätkö että minä joka olen Jumalan armost...
26998,-1,-1,0,-1,2,0,0,14,9,comments2018a.vrt 2653599,Ei näiltä happopäiltä jotka itseään kutsuvat s...


Let's pick only majority_value and sentence_text, and rename them to label and text.

In [6]:
df = df[['majority_value','sentence_text']]
df = df.rename({'sentence_text': 'text', 'majority_value': 'label'}, axis=1)
df

,label,text
0,1,- Tervetuloa skotlantiin...
1,0,"...... No, oikein sopiva sattumaha se vaan oli..."
2,0,40.
3,1,Kyseessä voi olla loppuelämäsi nainen.
4,1,Sinne vaan ocean clubiin iskemään!
...,...,...
26995,0,sais nauraa.
26996,-1,"Menkää töihin, Jonnekin muuannekin kun niihin ..."
26997,0,Ja tiedätkö että minä joka olen Jumalan armost...
26998,-1,Ei näiltä happopäiltä jotka itseään kutsuvat s...


Print a few metrics about token lengths (data samples sizes, i.e. how long each row is). Token can be a word, or punctuation, for example.

In [7]:
_ = check_token_length(df)

Token indices sequence length is longer than the specified maximum sequence length for this model (1163 > 512). Running this sequence through the model will result in indexing errors


Min length:  3
Mean length:  18.64462962962963
Median length:  15.0
Max length:  2338


Check if there's duplicate or NaN values left.

In [8]:
check_dupl_nan(df)

NaNs:  label    0
text     0
dtype: int64
Duplicates in label: 26997
Duplicates in text: 479


Let's remove duplicates (and NANs) in text column.

In [9]:
df = remove_dupl_nan(df)
df

,label,text
0,1,- Tervetuloa skotlantiin...
1,0,"...... No, oikein sopiva sattumaha se vaan oli..."
2,0,40.
3,1,Kyseessä voi olla loppuelämäsi nainen.
4,1,Sinne vaan ocean clubiin iskemään!
...,...,...
26516,0,sais nauraa.
26517,-1,"Menkää töihin, Jonnekin muuannekin kun niihin ..."
26518,0,Ja tiedätkö että minä joka olen Jumalan armost...
26519,-1,Ei näiltä happopäiltä jotka itseään kutsuvat s...


Finally, let's edit the label column from having distribution of [-1,0,1] into [2,0,1] for convenience's sake.

In [10]:
df.label = df.label.str.replace(r'-1','2')
df = df.astype({'label': 'int'})
df

,label,text
0,1,- Tervetuloa skotlantiin...
1,0,"...... No, oikein sopiva sattumaha se vaan oli..."
2,0,40.
3,1,Kyseessä voi olla loppuelämäsi nainen.
4,1,Sinne vaan ocean clubiin iskemään!
...,...,...
26516,0,sais nauraa.
26517,2,"Menkää töihin, Jonnekin muuannekin kun niihin ..."
26518,0,Ja tiedätkö että minä joka olen Jumalan armost...
26519,2,Ei näiltä happopäiltä jotka itseään kutsuvat s...


## 2. Creating dataset for pretraining (MLM)

Let's pick only the text column since labels are unnecessary for pretraining (MLM). Then let's create folds to divide the data into two subsets, validation and training sets. 

In [11]:
df_pretrain = df[['text']]
df_pretrain = create_kfolds(df_pretrain, num_splits=5, random_seed=2023)
df_pretrain

,text,kfold
0,- Tervetuloa skotlantiin...,3
1,"...... No, oikein sopiva sattumaha se vaan oli...",4
2,40.,3
3,Kyseessä voi olla loppuelämäsi nainen.,4
4,Sinne vaan ocean clubiin iskemään!,2
...,...,...
26516,sais nauraa.,0
26517,"Menkää töihin, Jonnekin muuannekin kun niihin ...",1
26518,Ja tiedätkö että minä joka olen Jumalan armost...,2
26519,Ei näiltä happopäiltä jotka itseään kutsuvat s...,4


Let's pick one fold for validation set and use the other folds for a training set. 

In [12]:
one_fold = 3
mlm_val = df_pretrain[df_pretrain.kfold==one_fold]
mlm_train = df_pretrain[df_pretrain.kfold!=one_fold]
mlm_val = mlm_val[['text']]
mlm_train = mlm_train[['text']]
mlm_val = mlm_val.reset_index(drop=True)
mlm_train = mlm_train.reset_index(drop=True)

In [13]:
print("Dataset lengths")
print(f"validation set: {len(mlm_val)}")
print(f"training set: {len(mlm_train)}")

Dataset lengths
validation set: 5304
training set: 21217


In [14]:
mlm_val

,text
0,- Tervetuloa skotlantiin...
1,40.
2,"""Koska naiset yleensä arvostavat miehessä enit..."
3,Työstä kuuluu maksaa palkkaa vai meinaatko ett...
4,"Tämähän ei maksa kaupungille mitään, mutta ope..."
...,...
5299,"Haluan myös kiittää lääkeyhtiöitä kuten Wyeth,..."
5300,Onhan teillä oma Temppelinnekin jo….
5301,Siinä mielessä seksuaalisuutta ei tarvita lisä...
5302,"Keskiansio, mitä se nyt uutisissa olikaan, mel..."


In [15]:
mlm_train

,text
0,"...... No, oikein sopiva sattumaha se vaan oli..."
1,Kyseessä voi olla loppuelämäsi nainen.
2,Sinne vaan ocean clubiin iskemään!
3,Itsekin pidän Keskustan kampanjointia ihan hyv...
4,"Kamppi, Kontula, Kluuvi"
...,...
21212,sais nauraa.
21213,"Menkää töihin, Jonnekin muuannekin kun niihin ..."
21214,Ja tiedätkö että minä joka olen Jumalan armost...
21215,Ei näiltä happopäiltä jotka itseään kutsuvat s...


Finally, let's save the datasets.

In [16]:
mlm_train.to_csv(cfg.data_folder+'mlm_train.csv', index=False)
mlm_val.to_csv(cfg.data_folder+'mlm_valid.csv', index=False)

## 3. Creating dataset for finetuning

For finetuning, let's divide the data into two subsets: training and testing sets. Let's try sklearn's train_test_split function this time.

Consider also other data splitting methods, such as stratified k fold [to preserve the percentage of samples for each class](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html).

In [17]:
training_data, testing_data = train_test_split(df, test_size=0.2, random_state=2023)
training_data = training_data.reset_index(drop=True)
testing_data = testing_data.reset_index(drop=True)

In [18]:
training_data

,label,text
0,2,– Ei...
1,0,"Kurkist kehtoon, kuinka kultaa Lapsi paljon tu..."
2,0,Entä jos miehellä ei enää seiso?
3,0,Eihän ala-ikäiset saa muutakaan tehdä ilman va...
4,0,"""Mies on naisen pää, koska Allah on toisia suo..."
...,...,...
21211,0,"En voi käyttä samaa nimeä, koska Jumanmi Keklr..."
21212,0,Metsästysharrastusta kait tarkoitettiin päättö...
21213,0,Monet näistä hurjannäköisistä roduista eivät o...
21214,0,Puolilämpimällä koneella kierrokset 2000:ssa h...


In [19]:
testing_data

,label,text
0,0,"Ensinnäkin, korvikkeen saa lämmittää mikrossa,..."
1,0,"En tiedä, miksi trollaat asialla, joka on help..."
2,0,Nyt todella tiedän mitä glamour elämä on...
3,0,Kissa oli saanut olla itse valitsemansa ajan e...
4,2,Mietippä rehellisesti tiedätkö oikeasti narsis...
...,...,...
5300,0,itse olen nelikymppinen ja aion sairaanhoitoal...
5301,0,"Niin siis ""kannattaako"" jonkin harvinaisemman ..."
5302,0,Mulle kävi niin että se vaan yhtäkkii laitto s...
5303,2,"""Kalle ei ole Turtolan kanssa keskustellutkaan..."


In [20]:
print("Testing data distribution")
_ = check_class_distribution(testing_data)
print("Training data distribution")
_ = check_class_distribution(training_data)

Testing data distribution
For label 0 there is 3956 data samples
For label 1 there is 560 data samples
For label 2 there is 789 data samples
Training data distribution
For label 0 there is 15549 data samples
For label 1 there is 2368 data samples
For label 2 there is 3299 data samples


Finally, let's save the datasets.

In [21]:
testing_data.to_csv(cfg.data_folder+'finetune_testset.csv', index=False)
training_data.to_csv(cfg.data_folder+'finetune_trainset.csv', index=False)